# Example of how to (visually) compare dynamic optimization algorithms

Visualization of comparison results is done using Lets-Plot library using [Lets-Plot Kotlin API](https://lets-plot.org/kotlin). You can see the latest release in JetBrain's [lets-plot-kotlin](https://github.com/JetBrains/lets-plot-kotlin) repository and check some starting examples in their [Lets-Plot Usage Guide](https://nbviewer.org/github/JetBrains/lets-plot-kotlin/blob/master/docs/guide/user_guide.ipynb). Also, see [some examples](https://nbviewer.org/github/JetBrains/lets-plot-kotlin/tree/master/docs/examples/jupyter-notebooks/) by specific release.

#### "Line Magics"

This (`%use lets-plot`) "line magic" will apply **Lets-Plot library descriptor**, which adds to your notebook all the boilerplate code necessary to create plots. `%useLatestDescriptors` is needed for function `ggtb()`, which enables zoom/pan interactivity on the plot.

In [86]:
%useLatestDescriptors
%use lets-plot

#### EARS dependency

Import all EARS classes needed to perform the comparison.

In [87]:
import org.um.feri.ears.algorithms.DummyAlgorithm
import org.um.feri.ears.algorithms.NumberAlgorithm
import org.um.feri.ears.benchmark.Benchmark
import org.um.feri.ears.examples.dynopt.AlgorithmPerformance
import org.um.feri.ears.examples.dynopt.GMPBBenchmark
import org.um.feri.ears.statistic.rating_system.RatingType
import org.um.feri.ears.visualization.rating.RatingIntervalPlot
import java.io.File

#### Set the configuration for the comparison using EARS

Define the directory path for storing algorithm results.

In [88]:
val algResultsDir =
    "D:${File.separator}" +
            "EDOLAB-MATLAB${File.separator}" +
            "Results${File.separator}" +
            "Comparison${File.separator}" +
            "CEC2024${File.separator}" +
            "GMPB_Peaks5_ChangeFrequency5000_D5_ShiftSeverity1_Environments100"

Configure benchmark settings and disable certain features.

In [89]:
Benchmark.printInfo = false
//DummyAlgorithm.readFromJson = false
val displayRatingChart = false

Create a list of algorithms to be compared.

In [90]:
val players = arrayListOf<NumberAlgorithm>(
    //DummyAlgorithm("ACFPSO", algResultsDir),
    DummyAlgorithm("AMPDE", algResultsDir),
    //DummyAlgorithm("AMPPSO", algResultsDir),
    //DummyAlgorithm("AMSO", algResultsDir),
    //DummyAlgorithm("AmQSO", algResultsDir),
    //DummyAlgorithm("CDE", algResultsDir),
    //DummyAlgorithm("CESO", algResultsDir),
    DummyAlgorithm("CPSO", algResultsDir),
    //DummyAlgorithm("CPSOR", algResultsDir),
    //DummyAlgorithm("DSPSO", algResultsDir),
    //DummyAlgorithm("DynDE", algResultsDir),
    //DummyAlgorithm("DynPopDE", algResultsDir),
    //DummyAlgorithm("FTMPSO", algResultsDir),
    //DummyAlgorithm("HmSO", algResultsDir),
    //DummyAlgorithm("IDSPSO", algResultsDir),
    //DummyAlgorithm("ImQSO", algResultsDir),
    //DummyAlgorithm("RPSO", algResultsDir),
    //DummyAlgorithm("SPSO_AP_AD", algResultsDir),
    //DummyAlgorithm("TMIPSO", algResultsDir),
    //DummyAlgorithm("mCMAES", algResultsDir),
    //DummyAlgorithm("mDE", algResultsDir),
    //DummyAlgorithm("mPSO", algResultsDir),
    //DummyAlgorithm("mQSO", algResultsDir),
    DummyAlgorithm("mjDE", algResultsDir),
    //DummyAlgorithm("psfNBC", algResultsDir)
)

Initialize objects that will store algorithms' performance, set the number of runs, and calculate the total number of problems.

In [91]:
val algPerf = players.map { AlgorithmPerformance(it.id) }.toMutableList() // Prepare objects for storing the algorithms data.

val runNumber = 31

val changeFrequency = 5000
val environmentNumber = 100
val sampleInterval = 100
val numOfProblems = (changeFrequency / sampleInterval) * environmentNumber + environmentNumber - 1

#### Perform the comparison using EARS

* Initialize the evaluation number and iterate through all problems.
* For each problem, create and execute benchmarks, then store and optionally display the results.
* Update the evaluation number based on the defined interval.

In [92]:
var evaluationNumber = sampleInterval
for (i in 0 until numOfProblems) {
    // Create and execute benchmarks for every evaluation defined with sampleInterval.
    val gmpbBenchmark = GMPBBenchmark(evaluationNumber).apply {
        setDisplayRatingCharts(false)
        addAlgorithms(players)
        run(runNumber)
    }

    val tournamentResults = gmpbBenchmark.tournamentResults

    if (displayRatingChart) {
        RatingIntervalPlot.displayChart(
            tournamentResults.players,
            RatingType.GLICKO2,
            "Rating Interval for Eval$evaluationNumber"
        )
    }

    // Store tournament results for every player (algorithm).
    tournamentResults.players.forEach { player ->
        algPerf.firstOrNull { it.name == player.id }?.apply {
            addRating(
                "mpbeval$evaluationNumber",
                player.glicko2Rating.rating,
                player.glicko2Rating.ratingDeviation
            )
        }
    }

    if (evaluationNumber % changeFrequency == 0) {
        // first evaluation after environment/problem change
        evaluationNumber++
    } else if (evaluationNumber % sampleInterval == 0) {
        // evaluation every 'sampleInterval'
        evaluationNumber += sampleInterval
    } else {
        // next evaluation after environment/problem change
        evaluationNumber = evaluationNumber + sampleInterval - 1
    }
}

TrueSkill One-On-One rating:
mjDE - Mean(mu)=19,27, Std-Dev(sigma)=1,22 RI=[16,83, 21,72]
AMPDE - Mean(mu)=18,8, Std-Dev(sigma)=1,13 RI=[16,54, 21,06]
CPSO - Mean(mu)=18,02, Std-Dev(sigma)=1,03 RI=[15,96, 20,07]

TrueSkill Free-For-All rating:
mjDE - Mean(mu)=26,94, Std-Dev(sigma)=1,24 RI=[24,46, 29,42]
AMPDE - Mean(mu)=25,39, Std-Dev(sigma)=1,11 RI=[23,16, 27,61]
CPSO - Mean(mu)=24,3, Std-Dev(sigma)=1,12 RI=[22,06, 26,53]

Glicko2 rating:
mjDE - Rating=1.645,6 RD=64,8 ro=0,06 RI=[1515,98, 1775,21]
AMPDE - Rating=1.491,9 RD=64,8 ro=0,06 RI=[1362,29, 1621,53]
CPSO - Rating=1.362,5 RD=64,8 ro=0,06 RI=[1232,88, 1492,11]

Game results:
mjDE [win=40, lose=22, draw=0]
	 Against:{CPSO=[win=22, lose=9, draw=0], AMPDE=[win=18, lose=13, draw=0]}
	 Problems:{GMPBEval100=[win=40, lose=22, draw=0]}
AMPDE [win=23, lose=24, draw=15]
	 Against:{CPSO=[win=10, lose=6, draw=15], mjDE=[win=13, lose=18, draw=0]}
	 Problems:{GMPBEval100=[win=23, lose=24, draw=15]}
CPSO [win=15, lose=32, draw=15]
	 Against:{

#### Visualize the results of dynamic optimization algorithms comparison

A function that generates a sequence of evaluation numbers, increasing by sampleInterval, and adds an extra (evaluation) number immediately after each environment change.

In [93]:
fun generateSequence(changeFrequency: Int, environmentNumber: Int, sampleInterval: Int = 1000): List<Int> {
    val result = mutableListOf<Int>()
    // generate numbers increasing by sampleInterval
    for (i in sampleInterval..changeFrequency * environmentNumber step sampleInterval) {
        result.add(i)
        if (i % changeFrequency == 0 && i != changeFrequency * environmentNumber) {
            result.add(i + 1)   // add the next (evaluation) number just after the change happens
        }
    }
    return result
}

Import necessary classes from the letsPlot library, determine the number of algorithms, generate a sequence of x-values, and create the base plot, mapping the evaluation numbers to the "eval" key.

In [94]:
import org.jetbrains.letsPlot.commons.values.Color

val numAlgorithms = algPerf.size
val xValues = generateSequence(changeFrequency, environmentNumber, sampleInterval)

var plot = letsPlot(mapOf("eval" to xValues))

For each algorithm, extract rating values and calculate the lower and upper bounds for rating deviations. Prepare data for plotting, including evaluation numbers, ratings, bounds, and algorithm labels. Add a line plot, a ribbon for rating deviations, and points to the plot for each algorithm.

In [95]:
for (i in 0 until numAlgorithms) {
    val yValues = algPerf[i].evalData.values.map { it.rating } // // extract the rating values for the current algorithm

    // calculate the lower and upper bounds for the rating deviations
    val lowerBounds = algPerf[i].evalData.values.map { it.rating - 2 * it.ratingDeviation }
    val upperBounds = algPerf[i].evalData.values.map { it.rating + 2 * it.ratingDeviation }

    // prepare data for the current algorithm
    val data = mapOf(
        "eval" to xValues,
        "rating" to yValues,
        "lowerBounds" to lowerBounds,
        "upperBounds" to upperBounds,
        "Algorithm" to List(yValues.size) { algPerf[i].name }  // label each algorithm
    )

    // add rating line to the plot (cycling through predefined colors)
    plot += geomLine(data = data) {
        x = "eval"
        y = "rating"
        color = "Algorithm"
    }

    // add rating deviation ribbon to the plot (using the same color with transparency)
    plot += geomRibbon(data = data, alpha = 0.2) {
        x = "eval"
        ymin = "lowerBounds"
        ymax = "upperBounds"
        color = "Algorithm"
        fill = "Algorithm"
    }

    // add points at the same locations as the line
    plot += geomPoint(data = data) {
        x = "eval"
        y = "rating"
        color = "Algorithm"
    }
}

Find the minimum and maximum ratings in the entire comparison if you want to zoom in on the plot.

In [96]:
val allRatings = algPerf.flatMap { it.evalData.values.map { it.rating } }
val minY = allRatings.minOrNull() ?: 0.0
val maxY = allRatings.maxOrNull() ?: 1.0

//plot += scaleYContinuous(limits = minY to maxY)

Define the range of x-values (evaluations) you want to show on the plot.

In [97]:
val minX = 1000
val maxX = 25000

plot += scaleXContinuous(
    breaks = xValues,
    labels = xValues.map { it.toInt().toString() }, // specify the tick marks   .take(valuesToShow)
    limits = minX to maxX
)

Optionally add vertical lines (at the evaluation where change happens).

In [98]:
val verticalLines = (changeFrequency..changeFrequency * environmentNumber step changeFrequency).toList()
verticalLines.forEach { evalPoint ->
    plot += geomVLine(xintercept = evalPoint, color = "grey", linetype = "dashed") {
        // you can customize the appearance here
    }
}

Define the plot's title, width, and height and show it.

In [103]:
val title = "Dynamic optimization algorithms ratings during optimization"
val width = 2500
val height = 750

plot += ggsize(width, height)
plot += ggtitle(title)
plot += ggtb()  // enable Zoom and Pan interactivity

plot.show()

Optionally save the plot in the file.

In [100]:
val filename = "CEC2024_GMPB_Peaks5_ChangeFrequency5000_D5_ShiftSeverity1_Environments100.svg"

//ggsave(plot, filename)